In [1]:
#DTS notebook

In [2]:
! pip install sklearn-pandas
! pip install pycox
! pip install lifelines

     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 41 kB 586 kB/s 
     |████████████████████████████████| 65 kB 4.3 MB/s 
     |████████████████████████████████| 357 kB 60.1 MB/s 
     |████████████████████████████████| 2.2 MB 47.2 MB/s 
     |████████████████████████████████| 121 kB 71.4 MB/s 
     |████████████████████████████████| 1.9 MB 55.4 MB/s 
     |████████████████████████████████| 348 kB 8.6 MB/s 
     |████████████████████████████████| 55 kB 4.5 MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4048 sha256=5d141b7fc3721e759c8b9f2a1588198c46cf8d948436f0bc75a3964ab9725364
  Stored in directory: /root/.cache/pip/wheels/9f/01/ee/1331593abb5725ff7d8c1333aee93a50a1c29d6ddda9665c9f
Successfully built autograd-gamma


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper 


import torch # For building the networks 
from torch import nn
import torch.nn.functional as F
import torchtuples as tt # Some useful functions

from pycox.datasets import metabric
from pycox.models import LogisticHazard,CoxPH
from pycox.models.loss import NLLLogistiHazardLoss, CoxPHLoss, NLLPMFLoss,NLLPCHazardLoss
from pycox.evaluation import EvalSurv

from lifelines.utils import concordance_index
from lifelines import CoxPHFitter
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

np.random.seed(1234)
_ = torch.manual_seed(1234)

In [ ]:
#Loose Integration Models 

In [ ]:
#Deepsurv

In [ ]:
#Experiment Deepsurv

def sinlge_data_experiment_ds(data,dtype,maxind,dropout_rate,netsize,n_batch,resultdf,bsvec):

  for ind in np.arange(1,maxind + 1): #go through all 20 train/test pairs
    np.random.seed(1234)
    torchseed = torch.manual_seed(1234)
    
    testind = np.array(pd.read_csv(ind_folder + 'testind' + str(ind)+ ".csv")["x"]) - 1 # minus 1 to match the array index in Python
    trainind = np.array(pd.read_csv(ind_folder + 'trainind' + str(ind)+ ".csv")["x"]) - 1

    print('start with seed' + str(ind))

    i = data.shape[1]

    df_train = data.loc[trainind,]
    df_test = data.loc[testind,]


    df_train["age"] = Age[trainind]
    df_train["stage"] = Stage[trainind]
    df_test["age"] = Age[testind]
    df_test["stage"] = Stage[testind]
    if ndata != "breast": 
      df_train["gender"] = Gender[trainind]
      df_test["gender"] = Gender[testind] 

    Y_train = Y[trainind]
    Y_test = Y[testind]
    E_train = E[trainind]
    E_test = E[testind]


    df_train["time"] = Y_train.astype('float32')
    df_train["event"] = E_train.astype('float32')
    df_train,df_val = train_test_split(df_train, test_size=0.2)
    df_test["time"] = Y_test.astype('float32')
    df_test["event"] = E_test.astype('float32')

    #cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
    var_num = i + 2
    if ndata != "breast": var_num = i + 3
    cols_leave = df_train.columns[0:var_num]
    print(cols_leave)
    #standardize = [([col], StandardScaler()) for col in cols_standardize]
    leave = [(col, None) for col in cols_leave]
    x_mapper = DataFrameMapper(leave)

    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')

    get_target = lambda df: (df['time'].values, df['event'].values)
    y_train = get_target(df_train)
    y_val = get_target(df_val)
    durations_test, events_test = get_target(df_test)
    val = x_val, y_val
    # We don't need to transform the test labels
    durations_test, events_test = get_target(df_test)
    durations_train, events_train = get_target(df_train)

    in_features = x_train.shape[1]
 
    out_features = 1
    dropout = dropout_rate


    net = torch.nn.Sequential(
        torch.nn.Linear(in_features, netsize*4),
        torch.nn.ReLU(),
        torch.nn.BatchNorm1d(netsize*4),
        torch.nn.Dropout(dropout),

        torch.nn.Linear(netsize*4, netsize),
        torch.nn.ReLU(),
        torch.nn.BatchNorm1d(netsize),
        torch.nn.Dropout(dropout),
        
        torch.nn.Linear(netsize, out_features)
    )


    model = CoxPH(net, tt.optim.Adam)
    batch_size = n_batch
    model.optimizer.set_lr(0.005)
    epochs = 500
    callbacks = [tt.callbacks.EarlyStopping()]
    verbose = True
    log = model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose,
                val_data=val, val_batch_size=batch_size)


    #training c-index
    _ = model.compute_baseline_hazards()
    surv = model.predict_surv_df(x_train)
    ev = EvalSurv(surv, durations_train, events_train, censor_surv='km')
    trainci = ev.concordance_td('antolini')
    print("Train CI:" + str(trainci))
    #testing c-index
    surv = model.predict_surv_df(x_test)
    ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
    testci = ev.concordance_td('antolini')
    print("Test CI:" + str(testci))

    time_grid = np.linspace(0, 3650, 100)
    testbs = ev.integrated_brier_score(time_grid) 
    bsvec = bsvec + np.array(ev.brier_score(time_grid).values)  

    resultdf.append((dtype,ind,
                     trainci,
                    testci,
                     testbs
                     ))
    


  return resultdf,bsvec


In [ ]:
#Breast Hyperparameter
#Colon Hyperparameter
ndata = "breast" #cancer type
main_folder = '/content/drive/My Drive/deepsurv_DTF/TCGA/'
type_folder = main_folder + ndata + "/" #change it to the path to deepsurv directory



fac_folder = type_folder + 'fac/'
ind_folder = type_folder + 'ind/'
data_folder = type_folder + ndata + '_clean/'
result_folder = main_folder + 'results/'


clinical = pd.read_csv(data_folder + ndata + "_clinic.csv") #read in clinical data


Y = np.array(clinical["OS.time"])
E = np.array(clinical["OS"])
Age = np.array(clinical["agegroup"])
Stage = np.array(clinical["stagegroup"])
if ndata != "breast": Gender = np.array(clinical["gender_n"])
grid = 100
interval = 100
#Genomic Datasets

#single data

dtype = "GE"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
ge = data.drop(data.columns[0], axis=1)

dtype = "CN"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
cn = data.drop(data.columns[0], axis=1)

dtype = "DM"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
dm = data.drop(data.columns[0], axis=1)

dtype = "CON"

data =pd.concat([ge, cn, dm], axis=1)
con = data


In [ ]:
#breast
resultdf = []
resultname = result_folder + ndata +"Deepsurv"+ "finalcollect.csv"

bsvec = np.zeros(100)
resultdf = []
datatype = "GE"
dropout_rate = 0.3
netsize = 32
n_batch = 128
resultdf,bsvec = sinlge_data_experiment_ds(ge,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Deepsurv"+ "BSovertime.csv", avgbs, delimiter=",")
bsvec = np.zeros(100)

datatype = "DM"
dropout_rate = 0.5
netsize = 32
n_batch = 128
resultdf,bsvec = sinlge_data_experiment_ds(dm,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
bsvec = np.zeros(100)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Deepsurv"+ "BSovertime.csv", avgbs, delimiter=",")

datatype = "CN"
dropout_rate = 0.5
netsize = 64
n_batch = 128
resultdf,bsvec = sinlge_data_experiment_ds(cn,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Deepsurv"+ "BSovertime.csv", avgbs, delimiter=",")

bsvec = np.zeros(100)

datatype = "CON"
dropout_rate = 0.5
netsize = 64
n_batch = 128
resultdf,bsvec = sinlge_data_experiment_ds(con,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Deepsurv"+ "BSovertime.csv", avgbs, delimiter=",")                                          

cols=['DataType','seed','trainCI','testCI','testIBS']
results = pd.DataFrame(resultdf,columns = cols)
avgtrainci = results['trainCI'].mean()
avgtestci = results['testCI'].mean()
avgtestibs = results['testIBS'].mean()
print("Avg TrainCI:" + str(avgtrainci))
print("Avg TestCI:" + str(avgtestci))
print("Avg TestIBS:" + str(avgtestibs))
results.to_csv(resultname,index=False,header = True)


In [ ]:

#Colon Hyperparameter
ndata = "colon"
#ndata = "breast"
main_folder = '/content/drive/My Drive/deepsurv_DTF/TCGA/'
type_folder = main_folder + ndata + "/" #change it to the path to deepsurv directory



fac_folder = type_folder + 'fac/'
ind_folder = type_folder + 'ind/'
data_folder = type_folder + ndata + '_clean/'
result_folder = main_folder + 'results/'


clinical = pd.read_csv(data_folder + ndata + "_clinic.csv") #read in clinical data


Y = np.array(clinical["OS.time"])
E = np.array(clinical["OS"])
Age = np.array(clinical["agegroup"])
Stage = np.array(clinical["stagegroup"])
if ndata != "breast": Gender = np.array(clinical["gender_n"])
grid = 100
interval = 100
#Genomic Datasets

#single data

dtype = "GE"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
ge = data.drop(data.columns[0], axis=1)

dtype = "CN"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
cn = data.drop(data.columns[0], axis=1)

dtype = "DM"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
dm = data.drop(data.columns[0], axis=1)

dtype = "CON"

data =pd.concat([ge, cn, dm], axis=1)
con = data


In [ ]:
resultdf = []
resultname = result_folder + ndata +"Deepsurv"+ "finalcollect.csv"


bsvec = np.zeros(100)
datatype = "GE"
dropout_rate = 0.5
netsize = 128
n_batch = 128
resultdf,bsvec = sinlge_data_experiment_ds(ge,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Deepsurv"+ "BSovertime.csv", avgbs, delimiter=",")


bsvec = np.zeros(100)
datatype = "DM"
dropout_rate = 0.3
netsize = 128
n_batch = 128
resultdf,bsvec = sinlge_data_experiment_ds(dm,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Deepsurv"+ "BSovertime.csv", avgbs, delimiter=",")

bsvec = np.zeros(100)
datatype = "CN"
dropout_rate = 0.5
netsize = 128
n_batch = 16
resultdf,bsvec = sinlge_data_experiment_ds(cn,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Deepsurv"+ "BSovertime.csv", avgbs, delimiter=",")

bsvec = np.zeros(100)
datatype = "CON"
dropout_rate = 0.5
netsize = 128
n_batch = 128
resultdf,bsvec = sinlge_data_experiment_ds(con,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Deepsurv"+ "BSovertime.csv", avgbs, delimiter=",")

cols=['DataType','seed','trainCI','testCI','testIBS']
results = pd.DataFrame(resultdf,columns = cols)
results.to_csv(resultname,index=False,header = True)
avgtrainci = results['trainCI'].mean()
avgtestci = results['testCI'].mean()
avgtestibs = results['testIBS'].mean()
print("Avg TrainCI:" + str(avgtrainci))
print("Avg TestCI:" + str(avgtestci))
print("Avg TestIBS:" + str(avgtestibs))








In [4]:
#Loghazard net

In [ ]:
def sinlge_data_experiment_logh(data,dtype,maxind,dropout_rate,netsize,n_batch,resultdf,bsvec):
  for ind in np.arange(1,maxind+1): #go through all 20 train/test pairs
    np.random.seed(1234)
    torchseed = torch.manual_seed(1234)
    testind = np.array(pd.read_csv(ind_folder + 'testind' + str(ind)+ ".csv")["x"]) - 1 # minus 1 to match the array index in Python
    trainind = np.array(pd.read_csv(ind_folder + 'trainind' + str(ind)+ ".csv")["x"]) - 1

    print('start with seed' + str(ind))

    i = data.shape[1]
    df_train = data.loc[trainind,]

    df_test = data.loc[testind,]


    df_train["age"] = Age[trainind]
    df_train["stage"] = Stage[trainind]
    df_test["age"] = Age[testind]
    df_test["stage"] = Stage[testind]
    if ndata != "breast": 
      df_train["gender"] = Gender[trainind]
      df_test["gender"] = Gender[testind] 

    Y_train = Y[trainind]
    Y_test = Y[testind]
    E_train = E[trainind]
    E_test = E[testind]


    df_train["time"] = Y_train.astype('float32')
    df_train["event"] = E_train.astype('float32')
    df_train,df_val = train_test_split(df_train, test_size=0.2)
    df_test["time"] = Y_test.astype('float32')
    df_test["event"] = E_test.astype('float32')

    #cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
    var_num = i + 2
    if ndata != "breast": var_num = i + 3
    cols_leave = df_train.columns[0:var_num]
    print(cols_leave)
    #standardize = [([col], StandardScaler()) for col in cols_standardize]
    leave = [(col, None) for col in cols_leave]
    x_mapper = DataFrameMapper(leave)
    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')

    num_durations =100
    labtrans = LogisticHazard.label_transform(num_durations)
    get_target = lambda df: (df['time'].values, df['event'].values)
    y_train = labtrans.fit_transform(*get_target(df_train))
    y_val = labtrans.transform(*get_target(df_val))


    train = (x_train, y_train)
    val = (x_val, y_val)    

    # We don't need to transform the test labels
    durations_test, events_test = get_target(df_test)
    durations_train, events_train = get_target(df_train)


    in_features = x_train.shape[1]
    out_features = labtrans.out_features
    batch_norm = True
    dropout = dropout_rate
    batch_size = n_batch

    n_node = netsize
    net1 = torch.nn.Sequential(
                    torch.nn.Linear(in_features, n_node*4),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm1d(n_node*4),
                    torch.nn.Dropout(dropout_rate),


                    torch.nn.Linear(netsize*4, netsize),
                    torch.nn.ReLU(),
                    torch.nn.BatchNorm1d(netsize),
                    torch.nn.Dropout(dropout_rate),

                    torch.nn.Linear(n_node, out_features)
                )
    model = LogisticHazard(net1, tt.optim.Adam(0.01), duration_index=labtrans.cuts)
    epochs = 500
    callbacks = [tt.cb.EarlyStopping()]

    log = model.fit(x_train, y_train, batch_size, epochs, callbacks, val_data=val)

    surv = model.predict_surv_df(x_train)
    ev = EvalSurv(surv, durations_train, events_train, censor_surv='km')


    time_grid = np.linspace(0, 3650, 100)

    trainci = ev.concordance_td('antolini')
    print(trainci)
    #testing c-index
    surv = model.predict_surv_df(x_test)
    ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')

    testci = ev.concordance_td('antolini')
    testbs = ev.integrated_brier_score(time_grid) 
    bsvec = bsvec + np.array(ev.brier_score(time_grid).values)  
    print(testci)
    print(testbs)

    resultdf.append((dtype,ind,
                     trainci,
                    testci,
                     testbs
                     ))
  

  return resultdf,bsvec


In [ ]:

#Colon Hyperparameter
ndata = "colon"
main_folder = '/content/drive/My Drive/deepsurv_DTF/TCGA/'
type_folder = main_folder + ndata + "/" #change it to the path to deepsurv directory



fac_folder = type_folder + 'fac/'
ind_folder = type_folder + 'ind/'
data_folder = type_folder + ndata + '_clean/'
result_folder = main_folder + 'results/'


clinical = pd.read_csv(data_folder + ndata + "_clinic.csv") #read in clinical data


Y = np.array(clinical["OS.time"])
E = np.array(clinical["OS"])
Age = np.array(clinical["agegroup"])
Stage = np.array(clinical["stagegroup"])
if ndata != "breast": Gender = np.array(clinical["gender_n"])
grid = 100
interval = 100
#Genomic Datasets

#single data

dtype = "GE"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
ge = data.drop(data.columns[0], axis=1)

dtype = "CN"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
cn = data.drop(data.columns[0], axis=1)

dtype = "DM"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
dm = data.drop(data.columns[0], axis=1)

dtype = "CON"

data =pd.concat([ge, cn, dm], axis=1)
con = data


In [ ]:
resultdf = []
resultname = result_folder + ndata +"LogHazard"+ "finalcollect.csv"


bsvec = np.zeros(100)
datatype = "GE"
dropout_rate = 0.1
netsize = 32
n_batch = 32
resultdf,bsvec = sinlge_data_experiment_logh(ge,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Loghazard"+ "BSovertime.csv", avgbs, delimiter=",")


bsvec = np.zeros(100)
datatype = "DM"
dropout_rate = 0.1
netsize = 32
n_batch = 16
resultdf,bsvec = sinlge_data_experiment_logh(dm,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Loghazard"+ "BSovertime.csv", avgbs, delimiter=",")

bsvec = np.zeros(100)
datatype = "CN"
dropout_rate = 0.1
netsize = 32
n_batch = 32
resultdf,bsvec = sinlge_data_experiment_logh(cn,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Loghazard"+ "BSovertime.csv", avgbs, delimiter=",")

bsvec = np.zeros(100)
datatype = "CON"
dropout_rate = 0.1
netsize = 32
n_batch = 16
resultdf,bsvec = sinlge_data_experiment_logh(con,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Loghazard"+ "BSovertime.csv", avgbs, delimiter=",")

cols=['DataType','seed','trainCI','testCI','testIBS']
results = pd.DataFrame(resultdf,columns = cols)
results.to_csv(resultname,index=False,header = True)
avgtrainci = results['trainCI'].mean()
avgtestci = results['testCI'].mean()
avgtestibs = results['testIBS'].mean()
print("Avg TrainCI:" + str(avgtrainci))
print("Avg TestCI:" + str(avgtestci))
print("Avg TestIBS:" + str(avgtestibs))








In [ ]:

ndata = "breast"
main_folder = '/content/drive/My Drive/deepsurv_DTF/TCGA/'
type_folder = main_folder + ndata + "/" #change it to the path to deepsurv directory



fac_folder = type_folder + 'fac/'
ind_folder = type_folder + 'ind/'
data_folder = type_folder + ndata + '_clean/'
result_folder = main_folder + 'results/'


clinical = pd.read_csv(data_folder + ndata + "_clinic.csv") #read in clinical data


Y = np.array(clinical["OS.time"])
E = np.array(clinical["OS"])
Age = np.array(clinical["agegroup"])
Stage = np.array(clinical["stagegroup"])
if ndata != "breast": Gender = np.array(clinical["gender_n"])
grid = 100
interval = 100
#Genomic Datasets

#single data

dtype = "GE"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
ge = data.drop(data.columns[0], axis=1)

dtype = "CN"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
cn = data.drop(data.columns[0], axis=1)

dtype = "DM"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
dm = data.drop(data.columns[0], axis=1)

dtype = "CON"

data =pd.concat([ge, cn, dm], axis=1)
con = data


In [ ]:
resultdf = []
resultname = result_folder + ndata +"LogHazard"+ "finalcollect.csv"


bsvec = np.zeros(100)
datatype = "GE"
dropout_rate = 0.3
netsize = 16
n_batch = 32
resultdf,bsvec = sinlge_data_experiment_logh(ge,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Loghazard"+ "BSovertime.csv", avgbs, delimiter=",")


bsvec = np.zeros(100)
datatype = "DM"
dropout_rate = 0.1
netsize = 16
n_batch = 32
resultdf,bsvec = sinlge_data_experiment_logh(dm,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Loghazard"+ "BSovertime.csv", avgbs, delimiter=",")

bsvec = np.zeros(100)
datatype = "CN"
dropout_rate = 0.1
netsize = 16
n_batch = 16
resultdf,bsvec = sinlge_data_experiment_logh(cn,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Loghazard"+ "BSovertime.csv", avgbs, delimiter=",")

bsvec = np.zeros(100)
datatype = "CON"
dropout_rate = 0.1
netsize = 32
n_batch = 16
resultdf,bsvec = sinlge_data_experiment_logh(con,datatype,20,dropout_rate,netsize,n_batch,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"Loghazard"+ "BSovertime.csv", avgbs, delimiter=",")

cols=['DataType','seed','trainCI','testCI','testIBS']
results = pd.DataFrame(resultdf,columns = cols)
results.to_csv(resultname,index=False,header = True)
avgtrainci = results['trainCI'].mean()
avgtestci = results['testCI'].mean()
avgtestibs = results['testIBS'].mean()
print("Avg TrainCI:" + str(avgtrainci))
print("Avg TestCI:" + str(avgtestci))
print("Avg TestIBS:" + str(avgtestibs))








In [ ]:
#DTS Encoded Survival Net

In [ ]:
def sinlge_data_experiment_dtsm(data,dtype,maxind,lw,es,en,ns,n_batch,resultdf,bsvec):
  for ind in np.arange(1,maxind+1): #go through all 20 train/test pairs
    np.random.seed(1234)
    torchseed = torch.manual_seed(1234)
    testind = np.array(pd.read_csv(ind_folder + 'testind' + str(ind)+ ".csv")["x"]) - 1 # minus 1 to match the array index in Python
    trainind = np.array(pd.read_csv(ind_folder + 'trainind' + str(ind)+ ".csv")["x"]) - 1

    print('start with seed' + str(ind))

    i = data.shape[1]
    df_train = data.loc[trainind,]

    df_test = data.loc[testind,]


    df_train["age"] = Age[trainind]
    df_train["stage"] = Stage[trainind]
    df_test["age"] = Age[testind]
    df_test["stage"] = Stage[testind]
    if ndata != "breast": 
      df_train["gender"] = Gender[trainind]
      df_test["gender"] = Gender[testind] 

    Y_train = Y[trainind]
    Y_test = Y[testind]
    E_train = E[trainind]
    E_test = E[testind]


    df_train["time"] = Y_train.astype('float32')
    df_train["event"] = E_train.astype('float32')
    df_train,df_val = train_test_split(df_train, test_size=0.2)
    df_test["time"] = Y_test.astype('float32')
    df_test["event"] = E_test.astype('float32')

    #cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
    var_num = i + 2
    if ndata != "breast": var_num = i + 3
    cols_leave = df_train.columns[0:var_num]
    print(cols_leave)
    #standardize = [([col], StandardScaler()) for col in cols_standardize]
    leave = [(col, None) for col in cols_leave]
    x_mapper = DataFrameMapper(leave)
    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')

    num_durations = 100
    labtrans = LogisticHazard.label_transform(num_durations)
    get_target = lambda df: (df['time'].values, df['event'].values)
    y_train_surv = labtrans.fit_transform(*get_target(df_train))
    y_val_surv = labtrans.transform(*get_target(df_val))

    train = tt.tuplefy(x_train, (y_train_surv, x_train))
    val = tt.tuplefy(x_val, (y_val_surv, x_val))

    # We don't need to transform the test labels
    durations_test, events_test = get_target(df_test)
    durations_train, events_train = get_target(df_train)
    
    class NetAESurv(nn.Module):
      def __init__(self, in_features, encoded_features, out_features):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_features, es*3), nn.ReLU(),
            nn.Linear(es*3, es*2), nn.ReLU(),
            nn.Linear(es*2, es*2), nn.ReLU(),
            nn.Linear(es*2, es*2), nn.ReLU(),
    
            nn.Linear(es*2, encoded_features),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoded_features, es*2), nn.ReLU(),
            nn.Linear(es*2, es*2), nn.ReLU(),
            nn.Linear(es*2, es*2), nn.ReLU(),
            nn.Linear(es*2, es*3), nn.ReLU(),
            nn.Linear(es*3, in_features),
        )
        self.surv_net = nn.Sequential(
            nn.Linear(encoded_features, ns*3), nn.ReLU(),
            nn.Linear(ns*3, ns*3), nn.ReLU(),
          #  nn.Dropout(0.5),
            nn.Linear(ns*3, ns*2), nn.ReLU(),
            nn.Linear(ns*2, ns*2), nn.ReLU(),
         #   nn.Dropout(0.5),
            nn.Linear(ns*2, ns), nn.ReLU(),
            nn.Linear(ns, out_features),
        )

      def forward(self, input):
        encoded = self.encoder(input)
        decoded = self.decoder(encoded)
        phi = self.surv_net(encoded)
        return phi, decoded

      def predict(self, input):
        # Will be used by model.predict later.
        # As this only has the survival output, 
        # we don't have to change LogisticHazard.
        encoded = self.encoder(input)
        return self.surv_net(encoded)

    in_features = x_train.shape[1]
    encoded_features = en
    out_features = labtrans.out_features
    net = NetAESurv(in_features, encoded_features, out_features)

    class LossAELogHaz(nn.Module):
      def __init__(self, alpha):
        super().__init__()
        assert (alpha >= 0) and (alpha <= 1), 'Need `alpha` in [0, 1].'
        self.alpha = alpha
        self.loss_surv = NLLLogistiHazardLoss() #NLLPMFLoss() #
        self.loss_ae = nn.MSELoss()
        
      def forward(self, phi, decoded, target_loghaz, target_ae):
        idx_durations, events = target_loghaz
        loss_surv = self.loss_surv(phi, idx_durations, events)
        loss_ae = self.loss_ae(decoded, target_ae)
        return self.alpha * loss_surv + (1 - self.alpha) * loss_ae

    loss = LossAELogHaz(lw)
    model = LogisticHazard(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts, loss=loss)

    metrics = dict(
    loss_surv = LossAELogHaz(1),
    loss_ae   = LossAELogHaz(0)
    )
    callbacks = [tt.cb.EarlyStopping()]

    batch_size = n_batch
    epochs = 100
    model.fit(*train, batch_size, epochs, callbacks, val_data=val, metrics=metrics)

    #training c-index
    surv = model.predict_surv_df(x_train)
    ev = EvalSurv(surv, durations_train, events_train, censor_surv='km')


    trainci = ev.concordance_td()
    print(trainci)
    #testing c-index
    surv = model.predict_surv_df(x_test)
    ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')

    testci = ev.concordance_td()
    time_grid = np.linspace(0, 3650, 100)

    testbs = ev.integrated_brier_score(time_grid)

    bsvec = bsvec + np.array(ev.brier_score(time_grid).values)  
    
    print(testci)

    resultdf.append((dtype,ind,
                    trainci,
                    testci,
                    testbs))


  return resultdf,bsvec


In [ ]:

#Colon Hyperparameter
ndata = "colon"
main_folder = '/content/drive/My Drive/deepsurv_DTF/TCGA/'
type_folder = main_folder + ndata + "/" #change it to the path to deepsurv directory



fac_folder = type_folder + 'fac/'
ind_folder = type_folder + 'ind/'
data_folder = type_folder + ndata + '_clean/'
result_folder = main_folder + 'results/'


clinical = pd.read_csv(data_folder + ndata + "_clinic.csv") #read in clinical data


Y = np.array(clinical["OS.time"])
E = np.array(clinical["OS"])
Age = np.array(clinical["agegroup"])
Stage = np.array(clinical["stagegroup"])
if ndata != "breast": Gender = np.array(clinical["gender_n"])
grid = 100
interval = 100
#Genomic Datasets

#single data

dtype = "GE"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
ge = data.drop(data.columns[0], axis=1)

dtype = "CN"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
cn = data.drop(data.columns[0], axis=1)

dtype = "DM"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
dm = data.drop(data.columns[0], axis=1)

dtype = "CON"

data =pd.concat([ge, cn, dm], axis=1)
con = data


In [ ]:
#colon
resultdf = []
resultname = result_folder + ndata +"DTSM"+ "finalcollect.csv"

bsvec = np.zeros(100)
datatype = "GE"
batlist = 128
lossweight = 0.3 
encodenum = 16
encodesize = 128
netsize = 64


resultdf,bsvec = sinlge_data_experiment_dtsm(ge,datatype,20,lossweight,encodesize,encodenum,netsize,batlist,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"DTSM"+ "BSovertime.csv", avgbs, delimiter=",")


bsvec = np.zeros(100)
datatype = "DM"
batlist = 16
lossweight = 0.3 
encodenum = 16
encodesize = 128
netsize = 64


resultdf,bsvec = sinlge_data_experiment_dtsm(dm,datatype,20,lossweight,encodesize,encodenum,netsize,batlist,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"DTSM"+ "BSovertime.csv", avgbs, delimiter=",")

bsvec = np.zeros(100)
datatype = "CN"
batlist = 16
lossweight = 0.3 
encodenum = 16
encodesize = 128
netsize = 64


resultdf,bsvec = sinlge_data_experiment_dtsm(cn,datatype,20,lossweight,encodesize,encodenum,netsize,batlist,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"DTSM"+ "BSovertime.csv", avgbs, delimiter=",")

bsvec = np.zeros(100)
datatype = "CON"
batlist = 128
lossweight = 0.3 
encodenum = 16
encodesize = 128
netsize = 64


resultdf,bsvec = sinlge_data_experiment_dtsm(con,datatype,20,lossweight,encodesize,encodenum,netsize,batlist,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"DTSM"+ "BSovertime.csv", avgbs, delimiter=",")


cols=['DataType','seed','trainCI','testCI','testIBS']
results = pd.DataFrame(resultdf,columns = cols)
results.to_csv(resultname,index=False,header = True)
avgtrainci = results['trainCI'].mean()
avgtestci = results['testCI'].mean()
avgtestibs = results['testIBS'].mean()
print("Avg TrainCI:" + str(avgtrainci))
print("Avg TestCI:" + str(avgtestci))
print("Avg TestIBS:" + str(avgtestibs))








In [ ]:
#breast
#Breast Hyperparameter

ndata = "breast"
main_folder = '/content/drive/My Drive/deepsurv_DTF/TCGA/'
type_folder = main_folder + ndata + "/" #change it to the path to deepsurv directory



fac_folder = type_folder + 'fac/'
ind_folder = type_folder + 'ind/'
data_folder = type_folder + ndata + '_clean/'
result_folder = main_folder + 'results/'


clinical = pd.read_csv(data_folder + ndata + "_clinic.csv") #read in clinical data


Y = np.array(clinical["OS.time"])
E = np.array(clinical["OS"])
Age = np.array(clinical["agegroup"])
Stage = np.array(clinical["stagegroup"])
if ndata != "breast": Gender = np.array(clinical["gender_n"])
grid = 100
interval = 100
#Genomic Datasets

#single data

dtype = "GE"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
ge = data.drop(data.columns[0], axis=1)

dtype = "CN"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
cn = data.drop(data.columns[0], axis=1)

dtype = "DM"
data = pd.read_csv(data_folder + dtype + "_" + ndata + "_t.csv") #read in single Genomic data type
dm = data.drop(data.columns[0], axis=1)

dtype = "CON"

data =pd.concat([ge, cn, dm], axis=1)
con = data

resultdf = []
resultname = result_folder + ndata +"DTSM"+ "finalcollect.csv"

bsvec = np.zeros(100)
datatype = "GE"
batlist = 16
lossweight = 0.5 
encodenum = 16
encodesize = 128
netsize = 64


resultdf,bsvec = sinlge_data_experiment_dtsm(ge,datatype,20,lossweight,encodesize,encodenum,netsize,batlist,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"DTSM"+ "BSovertime.csv", avgbs, delimiter=",")


bsvec = np.zeros(100)
datatype = "DM"
batlist = 128
lossweight = 0.3 
encodenum = 16
encodesize = 128
netsize = 64


resultdf,bsvec = sinlge_data_experiment_dtsm(dm,datatype,20,lossweight,encodesize,encodenum,netsize,batlist,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"DTSM"+ "BSovertime.csv", avgbs, delimiter=",")

bsvec = np.zeros(100)
datatype = "CN"
batlist = 16
lossweight = 0.3 
encodenum = 16
encodesize = 128
netsize = 64


resultdf,bsvec = sinlge_data_experiment_dtsm(cn,datatype,20,lossweight,encodesize,encodenum,netsize,batlist,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"DTSM"+ "BSovertime.csv", avgbs, delimiter=",")

bsvec = np.zeros(100)
datatype = "CON"
batlist = 16
lossweight = 0.5 
encodenum = 16
encodesize = 128
netsize = 64


resultdf,bsvec = sinlge_data_experiment_dtsm(con,datatype,20,lossweight,encodesize,encodenum,netsize,batlist,resultdf,bsvec)
#results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + datatype +"DTSM"+ "BSovertime.csv", avgbs, delimiter=",")


cols=['DataType','seed','trainCI','testCI','testIBS']
results = pd.DataFrame(resultdf,columns = cols)
results.to_csv(resultname,index=False,header = True)
avgtrainci = results['trainCI'].mean()
avgtestci = results['testCI'].mean()
avgtestibs = results['testIBS'].mean()
print("Avg TrainCI:" + str(avgtrainci))
print("Avg TestCI:" + str(avgtestci))
print("Avg TestIBS:" + str(avgtestibs))








In [ ]:
#DTS TF-Encoded Survival net

In [ ]:
#colon
ndata = "colon"
lossweight = 0.5
encodenum = 16
encodesize = 16
netsize = 128
batch = 32

toprank = 25

In [ ]:
#breast
ndata = "breast"
lossweight = 0.7
encodenum = 8
encodesize = 64
netsize = 16
batch = 32

toprank = 17

In [ ]:
main_folder = '/content/drive/My Drive/deepsurv_DTF/TCGA/'
type_folder = main_folder + ndata + "/" #change it to the path to deepsurv directory



fac_folder = type_folder + 'fac/'
ind_folder = type_folder + 'ind/'
data_folder = type_folder + ndata + '_clean/'
result_folder = main_folder + 'results/'


clinical = pd.read_csv(data_folder + ndata + "_clinic.csv") #read in clinical data


Y = np.array(clinical["OS.time"])
E = np.array(clinical["OS"])
Age = np.array(clinical["agegroup"])
Stage = np.array(clinical["stagegroup"])
if ndata != "breast": Gender = np.array(clinical["gender_n"])
grid = 100
interval = 100

In [ ]:
# Final model
#given hyperparameters of final model and make predictions
avglist = []
resultdf = []
bsvec = np.zeros(100)
resultname = result_folder + ndata + 'DTSM' + ".csv"


i = toprank

for ind in np.arange(1,21): #go through all 20 train/test pairs
  np.random.seed(1234)
  _ = torch.manual_seed(1234)
  lw = lossweight
  en = encodenum
  ns = netsize
  es = encodesize

  ba = batch

  print('seed:' + str(ind))
  print('rank: ' + str(i))

  df_train = pd.read_csv(fac_folder + 'train' + str(ind) + "rank" + str(i) + ".csv",header = None)
  df_test = pd.read_csv(fac_folder + 'test' + str(ind) + "rank"+ str(i) + ".csv",header= None)
  testind = np.array(pd.read_csv(ind_folder + 'testind' + str(ind)+ ".csv")["x"]) - 1 # minus 1 to match the array index in Python
  trainind = np.array(pd.read_csv(ind_folder + 'trainind' + str(ind)+ ".csv")["x"]) - 1

  df_train["age"] = Age[trainind]
  df_train["stage"] = Stage[trainind]
  if ndata != "breast":df_train["gender"] = Gender[trainind]
  df_test["age"] = Age[testind]
  df_test["stage"] = Stage[testind]
  if ndata != "breast":df_test["gender"] = Gender[testind]

  Y_train = Y[trainind]
  Y_test = Y[testind]
  E_train = E[trainind]
  E_test = E[testind]

  df_train["time"] = Y_train.astype('float32')
  df_train["event"] = E_train.astype('float32')
  df_train,df_val = train_test_split(df_train, test_size=0.2)
  df_test["time"] = Y_test.astype('float32')
  df_test["event"] = E_test.astype('float32')

  #cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
  var_num = i + 2
  if ndata != "breast": var_num = i + 3
  cols_leave = df_train.columns[0:var_num]
  print(cols_leave)
  #standardize = [([col], StandardScaler()) for col in cols_standardize]
  leave = [(col, None) for col in cols_leave]
  x_mapper = DataFrameMapper(leave)
  x_train = x_mapper.fit_transform(df_train).astype('float32')
  x_val = x_mapper.transform(df_val).astype('float32')
  x_test = x_mapper.transform(df_test).astype('float32')

  num_durations = interval
  labtrans = LogisticHazard.label_transform(num_durations)
  get_target = lambda df: (df['time'].values, df['event'].values)
  y_train_surv = labtrans.fit_transform(*get_target(df_train))
  y_val_surv = labtrans.transform(*get_target(df_val))

  train = tt.tuplefy(x_train, (y_train_surv, x_train))
  val = tt.tuplefy(x_val, (y_val_surv, x_val))

  # We don't need to transform the test labels
  durations_test, events_test = get_target(df_test)
  durations_train, events_train = get_target(df_train)
  
  class NetAESurv(nn.Module):
    def __init__(self, in_features, encoded_features, out_features):
      super().__init__()
      self.encoder = nn.Sequential(
          nn.Linear(in_features, es*3), nn.ReLU(),
          nn.Linear(es*3, es*2), nn.ReLU(),
          nn.Linear(es*2, es*2), nn.ReLU(),
          nn.Linear(es*2, es*2), nn.ReLU(),
  
          nn.Linear(es*2, encoded_features),
      )
      self.decoder = nn.Sequential(
          nn.Linear(encoded_features, es*2), nn.ReLU(),
          nn.Linear(es*2, es*2), nn.ReLU(),
          nn.Linear(es*2, es*2), nn.ReLU(),
          nn.Linear(es*2, es*3), nn.ReLU(),
          nn.Linear(es*3, in_features),
      )
      self.surv_net = nn.Sequential(
          nn.Linear(encoded_features, ns*3), nn.ReLU(),
          nn.Linear(ns*3, ns*3), nn.ReLU(),
          nn.Linear(ns*3, ns*2), nn.ReLU(),
          nn.Linear(ns*2, ns*2), nn.ReLU(),
          nn.Linear(ns*2, ns), nn.ReLU(),
          nn.Linear(ns, out_features),
      )

    def forward(self, input):
      encoded = self.encoder(input)
      decoded = self.decoder(encoded)
      phi = self.surv_net(encoded)
      return phi, decoded

    def predict(self, input):
      # Will be used by model.predict later.
      # As this only has the survival output, 
      # we don't have to change LogisticHazard.
      encoded = self.encoder(input)
      return self.surv_net(encoded)

  in_features = x_train.shape[1]
  encoded_features = en
  out_features = labtrans.out_features
  net = NetAESurv(in_features, encoded_features, out_features)

  class LossAELogHaz(nn.Module):
    def __init__(self, alpha):
      super().__init__()
      assert (alpha >= 0) and (alpha <= 1), 'Need `alpha` in [0, 1].'
      self.alpha = alpha
      self.loss_surv = NLLLogistiHazardLoss() #NLLPMFLoss() #
      self.loss_ae = nn.MSELoss()
      
    def forward(self, phi, decoded, target_loghaz, target_ae):
      idx_durations, events = target_loghaz
      loss_surv = self.loss_surv(phi, idx_durations, events)
      loss_ae = self.loss_ae(decoded, target_ae)
      return self.alpha * loss_surv + (1 - self.alpha) * loss_ae

  loss = LossAELogHaz(lw)
  model = LogisticHazard(net, tt.optim.Adam(0.01), duration_index=labtrans.cuts, loss=loss)

  metrics = dict(
  loss_surv = LossAELogHaz(1),
  loss_ae   = LossAELogHaz(0)
  )
  callbacks = [tt.cb.EarlyStopping()]

  batch_size = ba
  epochs = 500
  model.fit(*train, batch_size, epochs, callbacks, val_data=val, metrics=metrics)

  #training c-index
  surv = model.predict_surv_df(x_train)
  ev = EvalSurv(surv, durations_train, events_train, censor_surv='km')

  time_grid = np.linspace(durations_test.min(), durations_test.max(), grid)

  trainci = ev.concordance_td()
  print(trainci)
  #testing c-index
  surv = model.predict_surv_df(x_test)
  ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')

  testci = ev.concordance_td()

  #time_grid = np.linspace(durations_test.min(), durations_test.max(), grid)
  time_grid = np.linspace(0, 3650, 100)

  testbs = ev.integrated_brier_score(time_grid)
  bsvec = bsvec + np.array(ev.brier_score(time_grid).values)
  
  print(testci)

  resultdf.append((ind,i,
                  trainci,
                  testci,
                  testbs))


  cols=['seed','rank','trainCI','testCI','testIBS']

  results = pd.DataFrame(resultdf,columns = cols)


avgtrainci = results['trainCI'].mean()
avgtestci = results['testCI'].mean()
avgtestibs = results['testIBS'].mean()
print("Avg TrainCI:" + str(avgtrainci))
print("Avg TestCI:" + str(avgtrainci))
print("Avg TestIBS:" + str(avgtestibs))

results.to_csv(resultname,index=False,header = True)
avgbs = bsvec/20
np.savetxt(result_folder + ndata + "DTSM"+ "BSovertime.csv", avgbs, delimiter=",")


In [ ]:
#TF-LogisticHazard


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper 
import torch # For building the networks 
from torch import nn
import torch.nn.functional as F
import torchtuples as tt # Some useful functions
from pycox.datasets import metabric
from pycox.models import LogisticHazard
from pycox.models.loss import NLLLogistiHazardLoss, CoxPHLoss, NLLPMFLoss,NLLPCHazardLoss
from pycox.evaluation import EvalSurv
from lifelines.utils import concordance_index
from lifelines import CoxPHFitter
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

#set randomseed for both torch and numpy
#np.random.seed(1234)
#_ = torch.manual_seed(1234)

In [ ]:
#colon
ndata = "colon"
dropout = 0.3
batchsize = 32
nodenum = 16
toprank = 46

In [ ]:
#breast
ndata = "breast"
dropout = 0.1
batchsize = 32
nodenum = 64
toprank = 22

In [ ]:

main_folder = '/content/drive/My Drive/deepsurv_DTF/TCGA/'
type_folder = main_folder + ndata + "/" #change it to the path to deepsurv directory


fac_folder = type_folder + 'fac/'
ind_folder = type_folder + 'ind/'
data_folder = type_folder + ndata + '_clean/'
result_folder = main_folder + 'results/'


clinical = pd.read_csv(data_folder + ndata + "_clinic.csv") #read in clinical data


Y = np.array(clinical["OS.time"])
E = np.array(clinical["OS"])
Age = np.array(clinical["agegroup"])
Stage = np.array(clinical["stagegroup"])
if ndata != "breast": Gender = np.array(clinical["gender_n"])

In [ ]:
avglist = []
resultdf = []
bsvec = np.zeros(100)
resultname = result_folder + ndata + 'LogHazard' + ".csv"
interval = 100
grid = 100
droprate = dropout
n_batch = batchsize
n_node = nodenum
i = toprank


for ind in np.arange(1,21): #go through all 20 train/test pairs
  np.random.seed(1234)
  torchseed = torch.manual_seed(1234)


  print('seed:' + str(ind))
  print('rank: ' + str(i))

  df_train = pd.read_csv(fac_folder + 'train' + str(ind) + "rank" + str(i) + ".csv",header = None)
  df_test = pd.read_csv(fac_folder + 'test' + str(ind) + "rank"+ str(i) + ".csv",header= None)
  testind = np.array(pd.read_csv(ind_folder + 'testind' + str(ind)+ ".csv")["x"]) - 1 # minus 1 to match the array index in Python
  trainind = np.array(pd.read_csv(ind_folder + 'trainind' + str(ind)+ ".csv")["x"]) - 1

  df_train["age"] = Age[trainind]
  df_train["stage"] = Stage[trainind]
  if ndata != "breast": df_train["gender"] = Gender[trainind]
  df_test["age"] = Age[testind]
  df_test["stage"] = Stage[testind]
  if ndata != "breast": df_test["gender"] = Gender[testind]

  Y_train = Y[trainind]
  Y_test = Y[testind]
  E_train = E[trainind]
  E_test = E[testind]

  df_train["time"] = Y_train.astype('float32')
  df_train["event"] = E_train.astype('float32')
  df_train,df_val = train_test_split(df_train, test_size=0.2)
  df_test["time"] = Y_test.astype('float32')
  df_test["event"] = E_test.astype('float32')

  #cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
  var_num = i + 2
  if ndata != "breast": var_num = i + 3
  cols_leave = df_train.columns[0:var_num]
  print(cols_leave)
  #standardize = [([col], StandardScaler()) for col in cols_standardize]
  leave = [(col, None) for col in cols_leave]
  x_mapper = DataFrameMapper(leave)
  x_train = x_mapper.fit_transform(df_train).astype('float32')
  x_val = x_mapper.transform(df_val).astype('float32')
  x_test = x_mapper.transform(df_test).astype('float32')

  num_durations = interval
  labtrans = LogisticHazard.label_transform(num_durations)
  get_target = lambda df: (df['time'].values, df['event'].values)
  y_train = labtrans.fit_transform(*get_target(df_train))
  y_val = labtrans.transform(*get_target(df_val))


  train = (x_train, y_train)
  val = (x_val, y_val)    

  # We don't need to transform the test labels
  durations_test, events_test = get_target(df_test)
  durations_train, events_train = get_target(df_train)
  

  in_features = x_train.shape[1]
  out_features = labtrans.out_features


  net2 = torch.nn.Sequential(
      torch.nn.Linear(in_features, n_node),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(n_node),
      torch.nn.Dropout(droprate),
      
      torch.nn.Linear(n_node, n_node),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(n_node),
      torch.nn.Dropout(droprate),
      
      torch.nn.Linear(n_node, out_features)
  )

  model = LogisticHazard(net2, tt.optim.Adam(0.01), duration_index=labtrans.cuts)


  batch_size = n_batch
  epochs = 500
  callbacks = [tt.cb.EarlyStopping()]

  log = model.fit(x_train, y_train, batch_size, epochs, callbacks, val_data=val)

  surv = model.predict_surv_df(x_train)
  ev = EvalSurv(surv, durations_train, events_train, censor_surv='km')
  


  time_grid = np.linspace(durations_test.min(), durations_test.max(), grid)

  trainci = ev.concordance_td()
  print(trainci)
  #testing c-index
  surv = model.predict_surv_df(x_test)
  ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')

  testci = ev.concordance_td()

   #time_grid = np.linspace(durations_test.min(), durations_test.max(), grid)
  time_grid = np.linspace(0, 3650, 100)

  testbs = ev.integrated_brier_score(time_grid)
  bsvec = bsvec + np.array(ev.brier_score(time_grid).values)
  
  print(testci)

  resultdf.append((ind,i,
                  trainci,
                  testci,
                  testbs))


  cols=['seed','rank','trainCI','testCI','testIBS']

  results = pd.DataFrame(resultdf,columns = cols)


avgtrainci = results['trainCI'].mean()
avgtestci = results['testCI'].mean()
avgtestibs = results['testIBS'].mean()
print("Avg TrainCI:" + str(avgtrainci))
print("Avg TestCI:" + str(avgtrainci))
print("Avg TestIBS:" + str(avgtestibs))

results.to_csv(resultname,index=False,header = True)
np.savetxt(result_folder + ndata + "LogHazard"+ "BSovertime.csv", bsvec/20, delimiter=",")



In [ ]:
#TF-Deepsurv


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# For preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper 

import torch # For building the networks 
from torch import nn
import torch.nn.functional as F
import torchtuples as tt # Some useful functions

from pycox.datasets import metabric
from pycox.models import LogisticHazard
from pycox.models.loss import NLLLogistiHazardLoss
from pycox.evaluation import EvalSurv
from pycox.datasets import metabric
from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

import tensorflow as tf

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
## Survival Analysis using Keras
from keras.initializers import glorot_uniform
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, RMSprop
from keras.regularizers import l2
import keras.backend as K
from lifelines.utils import concordance_index
from lifelines import CoxPHFitter

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
np.random.seed(1234)  # for reproducibility
_ = torch.manual_seed(1234)


In [ ]:
#colon
ndata = "colon"
dropout = 0.3
batchsize = 32
nodenum = 16
toprank = 18

In [ ]:
#breast
ndata = "breast"
dropout = 0.3
batchsize = 128
nodenum = 16
toprank = 22

In [ ]:


main_folder = '/content/drive/My Drive/deepsurv_DTF/TCGA/'
type_folder = main_folder + ndata + "/" #change it to the path to deepsurv directory


fac_folder = type_folder + 'fac/'
ind_folder = type_folder + 'ind/'
data_folder = type_folder + ndata + '_clean/'
result_folder = main_folder + 'results/'


clinical = pd.read_csv(data_folder + ndata + "_clinic.csv") #read in clinical data


Y = np.array(clinical["OS.time"])
E = np.array(clinical["OS"])
Age = np.array(clinical["agegroup"])
Stage = np.array(clinical["stagegroup"])
if ndata != "breast": Gender = np.array(clinical["gender_n"])

In [ ]:

#given hyperparameters of final model and make predictions
avglist = []
resultdf = []
bsvec = np.zeros(100)
resultname = result_folder + ndata + 'Deepsurv' + ".csv"

i = toprank
n_node = nodenum
droprate = dropout
n_batch = batchsize


for ind in np.arange(1,21): #go through all 20 train/test pairs
  np.random.seed(1234)
  torchseed = torch.manual_seed(1234)

  print('start with seed' + str(ind))

  print('current rank: ' + str(i))
  df_train = pd.read_csv(fac_folder + 'train' + str(ind) + "rank" + str(i) + ".csv",header = None)
  df_test = pd.read_csv(fac_folder + 'test' + str(ind) + "rank"+ str(i) + ".csv",header= None)
  
  testind = np.array(pd.read_csv(ind_folder + 'testind' + str(ind)+ ".csv")["x"]) - 1 # minus 1 to match the array index in Python
  trainind = np.array(pd.read_csv(ind_folder + 'trainind' + str(ind)+ ".csv")["x"]) - 1

  df_train["age"] = Age[trainind]
  df_train["stage"] = Stage[trainind]
  if ndata != "breast":df_train["gender"] = Gender[trainind]
  df_test["age"] = Age[testind]
  df_test["stage"] = Stage[testind]
  if ndata != "breast":df_test["gender"] = Gender[testind] 


  Y_train = Y[trainind]
  Y_test = Y[testind]
  E_train = E[trainind]
  E_test = E[testind]


  df_train["time"] = Y_train.astype('float32')
  df_train["event"] = E_train.astype('float32')
  df_train,df_val = train_test_split(df_train, test_size=0.2)
  df_test["time"] = Y_test.astype('float32')
  df_test["event"] = E_test.astype('float32')
  #step2

  #cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
  var_num = i + 2
  if ndata != "breast": var_num = i + 3
  cols_leave = df_train.columns[0:var_num]
  print(cols_leave)
  #standardize = [([col], StandardScaler()) for col in cols_standardize]
  leave = [(col, None) for col in cols_leave]
  x_mapper = DataFrameMapper(leave)

  x_train = x_mapper.fit_transform(df_train).astype('float32')
  x_val = x_mapper.transform(df_val).astype('float32')
  x_test = x_mapper.transform(df_test).astype('float32')

  get_target = lambda df: (df['time'].values, df['event'].values)
  y_train = get_target(df_train)
  y_val = get_target(df_val)
  durations_test, events_test = get_target(df_test)
  val = x_val, y_val
  # We don't need to transform the test labels
  durations_test, events_test = get_target(df_test)
  durations_train, events_train = get_target(df_train)

  in_features = x_train.shape[1]
  out_features = 1
  dropout = droprate


  net = torch.nn.Sequential(
      torch.nn.Linear(in_features, n_node*4),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(n_node*4),
      torch.nn.Dropout(dropout),

      torch.nn.Linear(n_node*4, n_node*2),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(n_node*2),
      torch.nn.Dropout(dropout),

      torch.nn.Linear(n_node*2, n_node*2),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(n_node*2),
      torch.nn.Dropout(dropout),
      
      torch.nn.Linear(n_node*2, out_features)
  )


  model = CoxPH(net, tt.optim.Adam)
  batch_size = n_batch
  model.optimizer.set_lr(0.005)
  epochs = 500
  callbacks = [tt.callbacks.EarlyStopping()]
  verbose = True
  log = model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose,
              val_data=val, val_batch_size=batch_size)



  #training c-index
  _ = model.compute_baseline_hazards()
  surv = model.predict_surv_df(x_train)
  ev = EvalSurv(surv, durations_train, events_train, censor_surv='km')
  trainci = ev.concordance_td('antolini')
  print(trainci)
  #testing c-index
  surv = model.predict_surv_df(x_test)
  ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
  testci = ev.concordance_td('antolini')
  print(testci)


  #time_grid = np.linspace(durations_test.min(), durations_test.max(), grid)
  time_grid = np.linspace(0, 3650, 100)

  testbs = ev.integrated_brier_score(time_grid)
  bsvec = bsvec + np.array(ev.brier_score(time_grid).values)
  
  print(testci)

  resultdf.append((ind,i,
                  trainci,
                  testci,
                  testbs))


  cols=['seed','rank','trainCI','testCI','testIBS']

  results = pd.DataFrame(resultdf,columns = cols)


avgtrainci = results['trainCI'].mean()
avgtestci = results['testCI'].mean()
avgtestibs = results['testIBS'].mean()
print("Avg TrainCI:" + str(avgtrainci))
print("Avg TestCI:" + str(avgtrainci))
print("Avg TestIBS:" + str(avgtestibs))

results.to_csv(resultname,index=False,header = True)
np.savetxt(result_folder + ndata + "Deepsurv"+ "BSovertime.csv", bsvec/20, delimiter=",")
